In [25]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import torch

# torch.manual_seed(seed)是用来为CPU设置生成随机数的种子,方便下次复现实验结果
torch.manual_seed(0)
# torch.backends.cudnn.deterministic和torch.backends.cudnn.benchmark是用来控制PyTorch在使用cuDNN时的行为。cuDNN是一个用于深度神经网络的加速库，它可以提高PyTorch在GPU上的运行速度。
# torch.backends.cudnn.deterministic如果设置为True，那么每次运行时都会选择一个确定的算法，保证结果的可复现性。torch.backends.cudnn.benchmark如果设置为True，那么会在开始时花费一些时间来寻找最适合当前配置的高效算法，以提高运行效率。
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

# torchvision是一个用于计算机视觉的库，提供常用的模型、数据集和图像处理方法。
# torchvision.models是一个包含了预训练模型的模块,可以直接使用它们或者进行微调。
import torchvision.models as models
# torchvision.transforms包含了图像变换方法的模块，可以用它们来对图像进行裁剪、旋转、缩放等操作。
import torchvision.transforms as transforms
# torchvision.datasets包含了常见数据集的模块，可以用它们来加载和处理数据。
import torchvision.datasets as datasets
# torch.nn包含了神经网络相关类和函数的模块，可以用它们来构建和训练神经网络。
# torch.nn.functional包含了一些激活函数、损失函数和其他函数的子模块，例如ReLU、交叉熵、卷积等。
import torch.nn as nn
import torch.nn.functional as F
# torch.optim是一个包含了优化器相关类和函数的模块，可以用它们来更新神经网络的参数
import torch.optim as optim
# torch.autograd是一个包含了自动求导相关类和函数的模块，可以用它们来计算神经网络中参数的梯度
from torch.autograd import Variable
# torch.utils.data.dataset包含了数据集相关类和函数的子模块，可以用它们来自定义和管理数据集。
from torch.utils.data.dataset import Dataset
# nibabel是一个用于处理医学图像格式的库，例如NIfTI、DICOM等。可以用它们来读取、保存和显示医学图像。
# nibabel.viewers.OrthoSlicer3D()类来显示其三维切片视图
import nibabel as nib
from nibabel.viewers import OrthoSlicer3D


In [68]:
train_path = glob.glob('/kaggle/input/pet-data/��PETͼ������ͼ���Ԥ����ս����������/Train/*/*')
test_path = glob.glob('/kaggle/input/pet-data/��PETͼ������ͼ���Ԥ����ս����������/Test/*')

np.random.shuffle(train_path)
np.random.shuffle(test_path)
# 数据缓存
DATA_CACHE = {}

class XunFeiDataset(Dataset):
    def __init__(self, img_path, transform=None):
        # 定义数据集路径
        self.img_path = img_path
        if transform is not None:
            # 传入变换
            self.transform = transform
        else:
            # 未传入变换
            self.transform = None
    
    def __getitem__(self, index):
        # 如果数据在缓存中，直接从缓存中读取
        if self.img_path[index] in DATA_CACHE:
            img = DATA_CACHE[self.img_path[index]]
        else:
            # 否则重新加载
            img = nib.load(self.img_path[index]) 
            # 取前三个维度
            img = img.dataobj[:,:,:, 0]
            # 存入缓存
            DATA_CACHE[self.img_path[index]] = img
        
        # 在通道中随机选择50个索引            
        idx = np.random.choice(range(img.shape[-1]), 50)
        # 在一张图片中选择这些索引作为使用的通道
        img = img[:, :, idx]
        # 转化为浮点型
        img = img.astype(np.float32)
        # 对图片进行变换
        if self.transform is not None:
            img = self.transform(image = img)['image']
        # img的维度从(高度, 宽度, 通道)变为(通道, 高度, 宽度)，因为OpenCV和Pytorch对图像的维度顺序有不同的要求
        img = img.transpose([2,0,1])
        return img,torch.from_numpy(np.array(int('NC' in self.img_path[index])))
    
    def __len__(self):
        # 图片的张数
        return len(self.img_path)
        
import albumentations as A
train_loader = torch.utils.data.DataLoader(
    # 第一个位置是图片路径,第二个位置是变换
    XunFeiDataset(train_path[:-10],
            A.Compose([
            A.RandomRotate90(),
            A.RandomCrop(120, 120),
            A.HorizontalFlip(p=0.6),
            A.RandomContrast(p=0.6),
            A.RandomBrightnessContrast(p=0.5),
        ])
    ), 
    # 些参数的意思是：
    # 每个批次加载2个样本
    # 每个epoch开始时，打乱数据集的顺序
    # 使用1个工作进程来加载数据
    # 不将数据存储在锁页内存中
    batch_size=2, shuffle=True, num_workers=1, pin_memory=False
)

# 验证集取数据集的后十个
val_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path[-10:],
            A.Compose([
            A.RandomCrop(120, 120),
        ])
    ), batch_size=2, shuffle=False, num_workers=1, pin_memory=False
)

# 测试集
test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(test_path,
            A.Compose([
            A.RandomCrop(128, 128),
            A.HorizontalFlip(p=0.5),
            A.RandomContrast(p=0.5),
        ])
    ), batch_size=2, shuffle=False, num_workers=1, pin_memory=False
)


In [ ]:
class XunFeiNet(nn.Module):
    def __init__(self):
        super(XunFeiNet, self).__init__()
        model = models.resnet18(True)
        # `in_channels`：输入通道数，这里是50。
        # `out_channels`：输出通道数，这里是64。
        # `kernel_size`：卷积核大小，这里是(7, 7)。
        # `stride`：步长，这里是(2, 2)。
        # `padding`：填充大小，这里是(3, 3)。
        # `bias`：是否使用偏置项，这里是False。
        model.conv1 = torch.nn.Conv2d(50, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 2)
        self.resnet = model
    # 前向函数
    def forward(self, img):        
        out = self.resnet(img)
        return out

# 实例化
model = XunFeiNet()
# 将模型移动到cuda设备上,在GPU上进行计算。如果你的电脑有多个GPU，可以使用model.to(torch.device('cuda:0'))这样的语法来指定使用哪个GPU。如果你想在CPU上计算，你可以使用model.to('cpu')来将模型移回CPU。
model = model.to('cuda')
# 创建一个交叉熵损失函数，用于计算模型输出的概率分布和真实标签之间的差异。并将其移动到cuda设备上，以便在GPU上进行计算。
criterion = nn.CrossEntropyLoss().cuda()
# optimizer = torch.optim.AdamW(model.parameters(), 0.001)：创建一个AdamW优化器，用于更新模型的参数。0.001是学习率，表示每次更新参数时的步长。model.parameters()表示要优化的模型参数。
optimizer = torch.optim.AdamW(model.parameters(), 0.001)

In [ ]:
# 传入训练加载器,模型,损失函数,优化器
def train(train_loader, model, criterion, optimizer):
    # 训练
    model.train()
    # 训练损失
    train_loss = 0.0
    for i, (input, target) in enumerate(train_loader):
        # input = input.cuda(non_blocking=True)：将输入移动到cuda设备上，使用非阻塞模式，提高效率
        input = input.cuda(non_blocking=True)
        # target = target.cuda(non_blocking=True)：将目标移动到cuda设备上，使用非阻塞模式，提高效率
        target = target.cuda(non_blocking=True)
        # 用模型对输入预测得到输出
        output = model(input)
        loss = criterion(output, target)
        # 所得结果与目标的差异
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 优化器更新参数
        optimizer.step()
        # 如果为20的倍数,
        if i % 20 == 0:
            print("loss",loss.item())
        # 累加到训练误差上
        train_loss += loss.item()
    # 返回训练误差/训练集
    return train_loss/len(train_loader)
            
def validate(val_loader, model, criterion):
    # 评估模式
    model.eval()
    # 验证集准确率
    val_acc = 0.0
    
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            input = input.cuda()
            target = target.cuda()
            # compute output
            output = model(input)
            loss = criterion(output, target)
            
            val_acc += (output.argmax(1) == target).sum().item()
            
    return val_acc / len(val_loader.dataset)
    
for i  in range(3):
    print(f"第{i}次")
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc  = validate(val_loader, model, criterion)
    train_acc = validate(train_loader, model, criterion)
    print("train_loss",train_loss)
    print("tran_acc",train_acc)
    print("val_acc", val_acc)
    

In [70]:
def predict(test_loader, model, criterion):
    # 模型评估,这行代码的意思是将模型设置为评估模式，也就是关闭一些在训练时和评估时表现不同的层或部分，例如Dropout层和BatchNorm层。在进行推理或预测时，必须调用model.eval()，否则会导致不一致的结果。
    model.eval()
    # 验证准确率
    val_acc = 0.0
    # 预测值
    test_pred = []
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input = input.cuda()
            target = target.cuda()
            # 用模型对输入进行预测，得到输出
            output = model(input)
            # 将输出从cuda设备上移回cpu,并转换为numpy数组,然后追加到test_pred列表中,用于保存测试结果
            test_pred.append(output.data.cpu().numpy())
    # 将test_pred列表中的数组按照垂直方向（行顺序）堆叠起来，形成一个新的数组
    return np.vstack(test_pred)
    
pred = None
for _ in range(10):
    if pred is None:
        pred = predict(test_loader, model, criterion)
    else:
        pred += predict(test_loader, model, criterion)
submit = pd.DataFrame(
    {
        'uuid': [int(x.split('/')[-1][:-4]) for x in test_path],
        'label': pred.argmax(1)
})
# 将submit[‘label’]列中的值替换为另一组值，根据一个字典的映射关系。字典的键是原来的值，字典的值是替换后的值。例如，这里将1替换为’NC’，将0替换为’MCI’。
submit['label'] = submit['label'].map({1:'NC', 0: 'MCI'})
# 按照"uuid"排序
submit = submit.sort_values(by='uuid')
# 转化为csv文件
submit.to_csv('submit101.csv', index=None)
